In [2]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,PF,2022-07-18 22:40:41,-23.1203,-134.9692,74.66,75,14,11.39
1,1,Nikolskoye,RU,2022-07-18 22:40:41,59.7035,30.7861,55.80,92,59,4.63
2,2,Qixia,CN,2022-07-18 22:40:42,37.2975,120.8256,76.50,59,22,0.38
3,3,Saskylakh,RU,2022-07-18 22:39:55,71.9167,114.0833,44.19,80,100,13.53
4,4,Rosita,NI,2022-07-18 22:40:42,13.9268,-84.4032,78.17,96,100,3.58


In [4]:
# Get the data types.
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp]) #[max(0,temp) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)


In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [23]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Qixia,CN,2022-07-18 22:40:42,37.2975,120.8256,76.50,59,22,0.38
4,4,Rosita,NI,2022-07-18 22:40:42,13.9268,-84.4032,78.17,96,100,3.58
7,7,Zhicheng,CN,2022-07-18 22:40:43,30.2956,111.5047,78.30,92,96,3.51
11,11,Albany,US,2022-07-18 22:38:58,42.6001,-73.9662,77.72,87,100,4.00
12,12,Georgetown,MY,2022-07-18 22:40:45,5.4112,100.3354,82.33,89,40,3.44
14,14,Port-Gentil,GA,2022-07-18 22:37:56,-0.7193,8.7815,75.16,83,40,8.05
15,15,Atuona,PF,2022-07-18 22:40:45,-9.8000,-139.0333,77.65,67,62,16.11
18,18,Hilo,US,2022-07-18 22:37:57,19.7297,-155.0900,89.28,72,100,10.36
21,21,Lompoc,US,2022-07-18 22:40:46,34.6391,-120.4579,81.84,61,0,16.11
22,22,Bandarbeyla,SO,2022-07-18 22:39:32,9.4942,50.8122,76.03,77,83,33.71


In [24]:
preferred_cities_df.count()

City_ID       220
City          220
Country       220
Date          220
Lat           220
Lng           220
Max Temp      220
Humidity      220
Cloudiness    220
Wind Speed    220
dtype: int64

In [25]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Qixia,CN,76.50,37.2975,120.8256,
4,Rosita,NI,78.17,13.9268,-84.4032,
7,Zhicheng,CN,78.30,30.2956,111.5047,
11,Albany,US,77.72,42.6001,-73.9662,
12,Georgetown,MY,82.33,5.4112,100.3354,
14,Port-Gentil,GA,75.16,-0.7193,8.7815,
15,Atuona,PF,77.65,-9.8000,-139.0333,
18,Hilo,US,89.28,19.7297,-155.0900,
21,Lompoc,US,81.84,34.6391,-120.4579,
22,Bandarbeyla,SO,76.03,9.4942,50.8122,


In [26]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [27]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [30]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [31]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [32]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))